# 3. Read CSV Data and Basic Data Analysis #
Examples based on Chapter 2, "Spark: Definitive Guide: Big Data processing Made Simple"

In this example, sample flight data for 2010 to 2015 is processed and Simple **MAX**, **Top-N** and **Group-By** operations are performed against the data using Spark SQL.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook-3").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        config("spark.eventLog.enabled", "true").\
        config("spark.eventLog.dir", "file:///opt/workspace/events").\
        getOrCreate()     

In [2]:
# Read flight data in from CSV
flightData = spark.read.option("inferSchema", True).option("header", True).csv("/opt/workspace/datain/flight-data/*.csv")

## Spark SQL Views ##
A temporary table-view can be constructed on a Spark data-frame with the `createOrReplaceTempView()` method.  

This allows the data-frame data to be analysed using ANSI SQL via the temporary table construct, which is an in-memory non-persistant view of the data built on the Spark data frame.

In [3]:
flightData.createOrReplaceTempView("flight_data")

#### Group-By Example ####
Query the data-frame and count the number of flights grouped by destination country.  
Using Spark SQL to query the temporary table `flight_data` creates another Spark data-frame (in this case, "`results`")

In [4]:
results = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data
GROUP BY DEST_COUNTRY_NAME
""")

In [5]:
# View the execution plan for running this query.  
results.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#10, 200)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#10] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/opt/workspace/datain/flight-data/2015-summary.csv, file:/opt/workspace/da..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


#### Max-Value Example ####

In [6]:
# Get max flight destination from flights using Spark SQL
spark.sql("SELECT max(count) from flight_data").take(1)

[Row(max(count)=370002)]

In [7]:
# Pyspark / Python equiv
from pyspark.sql.functions import max
flightData.select(max("count")).take(1)

[Row(max(count)=370002)]

#### Top-N Example ####

In [8]:
# Find top 5 destinations
top_5_dest = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")
top_5_dest.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|          2348280|
|           Canada|            49052|
|           Mexico|            38075|
|   United Kingdom|            10946|
|            Japan|             9205|
+-----------------+-----------------+



In [9]:
# Find top 5 destinations - DataFrame syntax
from pyspark.sql.functions import desc
flightData.groupBy("DEST_COUNTRY_NAME").sum("count")\
    .withColumnRenamed("sum(count)", "destination_total").sort(desc("destination_total"))\
    .limit(5).show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|          2348280|
|           Canada|            49052|
|           Mexico|            38075|
|   United Kingdom|            10946|
|            Japan|             9205|
+-----------------+-----------------+



#### Show Execution Path ####
Aggregation happens in two parts - in the partial_sum and sum calls - this is because summing a list of numbers is commutative* and Spark can perform the sum partition-by-partition.

\* commutative - " condition that a group of quantities connected by operators gives the same result whatever the order of the quantities involved, e.g. a × b = b × a."


In [10]:
spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""").explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[aggOrder#74L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#10,destination_total#72L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[sum(cast(count#12 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#10, 200)
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[partial_sum(cast(count#12 as bigint))])
         +- *(1) FileScan csv [DEST_COUNTRY_NAME#10,count#12] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/opt/workspace/datain/flight-data/2015-summary.csv, file:/opt/workspace/da..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


In [11]:
flightData.groupBy("DEST_COUNTRY_NAME").sum("count")\
    .withColumnRenamed("sum(count)", "destination_total").sort(desc("destination_total"))\
    .limit(5).explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#87L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#10,destination_total#87L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[sum(cast(count#12 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#10, 200)
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[partial_sum(cast(count#12 as bigint))])
         +- *(1) FileScan csv [DEST_COUNTRY_NAME#10,count#12] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/opt/workspace/datain/flight-data/2015-summary.csv, file:/opt/workspace/da..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>



*Exactly the same execution path is used*, whether performing the operation via Spark SQL or on the data-frame API using pyspark.

In [12]:
spark.stop()